# Window Functions and Rolling Totals in Oracle SQL

### Create Sample Sales Table
We create a simple sales table with orders, products, and amounts to demonstrate window functions.

In [ ]:
CREATE TABLE sales (
    order_id    NUMBER,
    order_date  DATE,
    product     VARCHAR2(50),
    amount      NUMBER
);

INSERT INTO sales VALUES (1, DATE '2025-01-05', 'Laptop', 1000);
INSERT INTO sales VALUES (2, DATE '2025-01-10', 'Phone',  500);
INSERT INTO sales VALUES (3, DATE '2025-01-20', 'Tablet', 300);
INSERT INTO sales VALUES (4, DATE '2025-02-02', 'Laptop', 1200);
INSERT INTO sales VALUES (5, DATE '2025-02-05', 'Phone',  700);
INSERT INTO sales VALUES (6, DATE '2025-02-15', 'Tablet', 400);

COMMIT;

### View Data
Check the contents of the sales table.

In [ ]:
SELECT * FROM sales ORDER BY order_date;

### Monthly Totals using GROUP BY
Classic aggregation to find total sales per month.

In [ ]:
SELECT TRUNC(order_date, 'MM') AS order_month,
       SUM(amount) AS monthly_total
FROM sales
GROUP BY TRUNC(order_date, 'MM')
ORDER BY order_month;

### Monthly Totals using Window Functions
Show monthly totals alongside each row without collapsing rows.

In [ ]:
SELECT order_id, order_date, product, amount,
       SUM(amount) OVER (PARTITION BY TRUNC(order_date,'MM')) AS monthly_total
FROM sales
ORDER BY order_date;

### Rolling Monthly Totals
Use window frame clause to compute running totals within each month.

In [ ]:
SELECT order_id, order_date, product, amount,
       SUM(amount) OVER (
         PARTITION BY TRUNC(order_date,'MM')
         ORDER BY order_date, order_id
         ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS running_total
FROM sales;

### Using LEAD Function
The `LEAD` function allows you to look ahead at the value of a column in the next row within the same partition.
Here, we find the **next order's amount** within the same month, along with the monthly total.

In [ ]:
SELECT order_id,
       order_date,
       product,
       amount,
       LEAD(amount, 1) OVER (
         PARTITION BY TRUNC(order_date,'MM')
         ORDER BY order_date, order_id
       ) AS next_order_amount,
       SUM(amount) OVER (PARTITION BY TRUNC(order_date,'MM')) AS monthly_total
FROM sales
ORDER BY order_date;

### Using LAG Function
The `LAG` function allows you to look back at the value of a column in the previous row within the same partition.
Here, we find the **previous order's amount** within the same month, along with the monthly total.

In [ ]:
SELECT order_id,
       order_date,
       product,
       amount,
       LAG(amount, 1) OVER (
         PARTITION BY TRUNC(order_date,'MM')
         ORDER BY order_date, order_id
       ) AS previous_order_amount,
       SUM(amount) OVER (PARTITION BY TRUNC(order_date,'MM')) AS monthly_total
FROM sales
ORDER BY order_date;